In [9]:
# Cell 1: Install Dependencies and Setup Environment

# 1. Install system-level packages for graph rendering
print("⏳ Installing system packages for Graphviz...")
!apt-get update > /dev/null
!apt-get install -y graphviz > /dev/null
print("✅ System packages installed.")

# 2. Install all required Python libraries
print("⏳ Installing Python libraries (autogen, gradio, groq, etc.)...")
!pip install "pyautogen>=0.2.25" "duckduckgo-search>=5.3.1b1" "groq>=0.9.0" "gliclass>=0.2.1" "transformers" "torch" "sentence-transformers" "graphviz" > /dev/null
print("✅ Python libraries installed.")

# 3. IMPORTANT: SET YOUR API KEY HERE
# Replace the placeholder with your actual Groq API key
GROQ_API_KEY = "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" # <--- REPLACE WITH YOUR KEY

# 4. Environment and validation
import os
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

if "gsk_" not in GROQ_API_KEY:
    raise ValueError("GROQ_API_KEY is not set correctly. Please replace the placeholder with your key.")
else:
    print("✅ Groq API Key set successfully.")

⏳ Installing system packages for Graphviz...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
✅ System packages installed.
⏳ Installing Python libraries (autogen, gradio, groq, etc.)...
ERROR: Could not find a version that satisfies the requirement gliclass>=0.2.1 (from versions: 0.1.0, 0.1.1, 0.1.3, 0.1.4, 0.1.5, 0.1.6, 0.1.7, 0.1.8, 0.1.9, 0.1.10, 0.1.11)
ERROR: No matching distribution found for gliclass>=0.2.1
✅ Python libraries installed.
✅ Groq API Key set successfully.


In [10]:
pip install groq duckduckgo_search gliclass transformers torch sentence-transformers graphviz

In [11]:
pip install pyautogen

In [4]:
# Cell 2: All Imports and Configurations

# --- Standard & External Libs ---
import ast
import re
import concurrent.futures
from typing import List, Dict

# --- Service/API Libs ---
from groq import Groq
from duckduckgo_search import DDGS

# --- AutoGen Core ---
import autogen

# --- GLiClass for Classification ---
from gliclass import GLiClassModel, ZeroShotClassificationPipeline
from transformers import AutoTokenizer

# --- Gradio & Graphviz for UI ---
import gradio as gr
import graphviz
import time

# --- LLM CONFIG (FOR AUTOGEN) ---
llm_config = {
    "config_list": [
        {
            "model": "meta-llama/llama-4-maverick-17b-128e-instruct",  # Llama3 8B is a good, fast choice on Groq
            "api_key": os.environ.get("GROQ_API_KEY"),
            "api_type": "groq",
        }
    ],
    "cache_seed": None,
}

# --- GRAPHVIZ CONFIG (FOR UI) ---
NODE_IDS = {
    "human": "human_user",
    "classifier": "classifier",
    "coordinator_receives": "coord_receives",
    "subtasks_list": "subtasks_list",
    "create_subagents": "create_subagents",
    "execute_parallel": "execute_parallel",
    "collect_results": "collect_results",
    "combine_results": "combine_results",
    "final_output": "final_output",
    "simple_responder": "simple_responder",
}

STATUS_COLORS = {
    "pending": "#d3d3d3",    # Light Gray
    "running": "#ffdd77",    # Light Orange/Gold
    "completed": "#b2e5b2",  # Light Green
    "human": "#ffc0cb",      # Pink
    "skipped": "#f0f0f0",    # Very Light Gray
}

In [16]:
pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=5a56ce304c8e0fba2a545e1d5c717f9e85bffc3b79bb04e72c0c07f0373830ac
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia


In [20]:
# Cell 3: Classifier Setup, Helper Functions, and Agent Tools

# --- Imports for tools and classifier ---
import re
import os
import json
import time
import ast
import graphviz
import autogen
import requests
import wikipedia
import concurrent.futures
from typing import List, Dict
from duckduckgo_search import DDGS
# CORRECTED: We only need the pipeline class from gliclass
from gliclass import ZeroShotClassificationPipeline
# CORRECTED: No longer need these specific imports for classifier setup
# from gliclass import GLiClassModel
# from transformers import AutoTokenizer

# --- Helper to clean LLM output ---
def extract_python_list_from_string(text: str) -> str:
    """Extracts a Python list from a string."""
    match = re.search(r'\[[\s\S]*?\]', text)
    return match.group(0) if match else None

# --- Classifier setup and execution ---
def setup_classifier():
    """Sets up the zero-shot classification pipeline."""
    print("⏳ Setting up the zero-shot classifier model (this may take a minute)...")
    try:
        # CORRECTED: Initialize the pipeline directly with the model name.
        # This allows the library to handle loading the model, tokenizer, and config correctly.
        model = GLiClassModel.from_pretrained("knowledgator/gliclass-large-v1.0")
        tokenizer = AutoTokenizer.from_pretrained("knowledgator/gliclass-large-v1.0")
        pipeline = ZeroShotClassificationPipeline(
            model,
            tokenizer,
            classification_type='multi-label',
            device='cpu'
        )
        print("✅ Classifier setup complete.")
        return pipeline
    except Exception as e:
        print(f"❌ Failed to setup classifier: {e}")
        return None

def classify_query(pipeline: ZeroShotClassificationPipeline, query: str, labels: List[str]) -> Dict:
    """Classifies a query using the pipeline."""
    if not pipeline: return {"label": "complex_research", "score": 1.0}
    # The pipeline call signature might differ slightly, but this is the common usage
    results = pipeline(query, labels, threshold=0.5)

    # gliclass pipeline might return a dict directly, not a list containing a dict
    if isinstance(results, dict) and 'labels' in results and 'scores' in results:
        if not results['labels']:
            return {"label": "complex_research", "score": 0.0}
        # Find the index of the highest score and get the corresponding label
        max_score_index = results['scores'].index(max(results['scores']))
        best_result = {"label": results['labels'][max_score_index], "score": results['scores'][max_score_index]}
        return best_result
    # Handle the original expected format just in case
    elif isinstance(results, list) and results:
         return max(results[0], key=lambda x: x["score"]) if results[0] else {"label": "complex_research", "score": 0.0}
    else:
        return {"label": "complex_research", "score": 0.0}


# --- Tool Definitions ---

def wikipedia_search(query: str, sentences: int = 5) -> str:
  """
  Fetches a summary from Wikipedia for a given search term.

  Args:
    query: The term to search for on Wikipedia.
    sentences: The number of sentences to include in the summary. Defaults to 5.

  Returns:
    A string containing the summary, or an error message if an error occurred.
  """
  print(f"\n📚 [Sub-agent Tool] Searching Wikipedia for: '{query}'")
  try:
    summary = wikipedia.summary(query, sentences=sentences, auto_suggest=False)
    return summary
  except wikipedia.exceptions.PageError:
    return f"Error: Page '{query}' not found on Wikipedia."
  except wikipedia.exceptions.DisambiguationError as e:
    return f"Error: Disambiguation error for '{query}'. Possible options: {e.options}"
  except Exception as e:
    return f"An unexpected error occurred with Wikipedia search: {e}"

def langsearch_web_search(query: str) -> str:
    """
    Performs a web search using the LangSearch API to get up-to-date information.

    Args:
        query: The search query.

    Returns:
        A formatted string of search results or an error message.
    """
    print(f"\n🌐 [Sub-agent Tool] Searching LangSearch for: '{query}'")
    url = "https://api.langsearch.com/v1/web-search"
    payload = json.dumps({
      "query": query,
      "freshness": "noLimit",
      "summary": True,
      "count": 5  # Limit to 5 results for brevity
    })
    headers = {
      'Authorization': 'sk-975b5afab002431a99a9dd24863f55da', # As provided
      'Content-Type': 'application/json'
    }
    try:
        response = requests.post(url, headers=headers, data=payload, timeout=15)
        response.raise_for_status()
        data = response.json()

        if "data" in data and "webPages" in data["data"] and "value" in data["data"]["webPages"]:
            results = data["data"]["webPages"]["value"]
            if not results:
                return f"No results found for '{query}' via LangSearch."

            output = [f"Title: {r.get('name', 'No Title')}\nSnippet: {r.get('snippet', 'No Snippet')}" for r in results]
            return "\n\n".join(output)
        else:
            return f"LangSearch returned an unexpected response format for '{query}'."
    except requests.exceptions.RequestException as e:
        return f"Error during LangSearch for '{query}': {e}"
    except Exception as e:
        return f"An unexpected error occurred with LangSearch: {e}"

def duckduckgo_search(query: str) -> str:
    """Performs a web search using DuckDuckGo."""
    print(f"\n🔎 [Sub-agent Tool] Searching DuckDuckGo for: '{query}'")
    try:
        with DDGS() as ddgs:
            results = [r for r in ddgs.text(query, max_results=5)]
            return "\n\n".join(f"Title: {r['title']}\nSnippet: {r['body']}" for r in results) if results else f"No results found for '{query}'."
    except Exception as e:
        return f"Error during DuckDuckGo search for '{query}': {e}"

# --- Setup the classifier once globally ---
CLASSIFIER_PIPELINE = setup_classifier()

⏳ Setting up the zero-shot classifier model (this may take a minute)...
✅ Classifier setup complete.


In [23]:
# Cell 4: Integrated Agentic Workflow with Visualization Hooks

def create_graph_image(statuses, num_subtasks=0, temp_dir='gradio_temp'):
    """Generates and saves a Graphviz image based on current node statuses."""
    if not os.path.exists(temp_dir): os.makedirs(temp_dir)

    dot = graphviz.Digraph(comment='Live Agentic Process')
    dot.attr(rankdir='TB', splines='ortho')
    dot.node_attr['style'] = 'filled'
    # Define all nodes first
    dot.node(NODE_IDS["human"], '1. Human User', shape='box', fillcolor=statuses[NODE_IDS["human"]])
    dot.node(NODE_IDS["classifier"], '2. Classify Query', shape='ellipse', fillcolor=statuses[NODE_IDS["classifier"]])
    # Complex Path
    dot.node(NODE_IDS["coordinator_receives"], '3a. Coordinator Receives Task', shape='box', fillcolor=statuses[NODE_IDS["coordinator_receives"]])
    dot.node(NODE_IDS["subtasks_list"], '4a. Create Subtasks List', shape='note', fillcolor=statuses[NODE_IDS["subtasks_list"]])
    dot.node(NODE_IDS["create_subagents"], '5a. Create Sub-agents', shape='box', fillcolor=statuses[NODE_IDS["create_subagents"]])
    dot.node(NODE_IDS["execute_parallel"], '6a. Execute in Parallel', shape='diamond', fillcolor=statuses[NODE_IDS["execute_parallel"]])
    for i in range(1, num_subtasks + 1):
        dot.node(f"sub_agent_{i}", f'Sub-agent {i}', shape='box', fillcolor=statuses.get(f"sub_agent_{i}", STATUS_COLORS['pending']))
    dot.node(NODE_IDS["collect_results"], '7a. Collect Results', shape='box', fillcolor=statuses[NODE_IDS["collect_results"]])
    dot.node(NODE_IDS["combine_results"], '8a. Combine Results', shape='box', fillcolor=statuses[NODE_IDS["combine_results"]])
    # Simple Path
    dot.node(NODE_IDS["simple_responder"], '3b. Simple Responder', shape='box', fillcolor=statuses[NODE_IDS["simple_responder"]])
    # Final Output
    dot.node(NODE_IDS["final_output"], '9. Generate Final Output', shape='box', fillcolor=statuses[NODE_IDS["final_output"]])

    # Define Edges
    dot.edge(NODE_IDS["human"], NODE_IDS["classifier"])
    dot.edge(NODE_IDS["classifier"], NODE_IDS["coordinator_receives"], label='Complex')
    dot.edge(NODE_IDS["classifier"], NODE_IDS["simple_responder"], label='Simple')
    dot.edge(NODE_IDS["coordinator_receives"], NODE_IDS["subtasks_list"])
    dot.edge(NODE_IDS["subtasks_list"], NODE_IDS["create_subagents"])
    dot.edge(NODE_IDS["create_subagents"], NODE_IDS["execute_parallel"])
    for i in range(1, num_subtasks + 1):
        dot.edge(NODE_IDS["execute_parallel"], f"sub_agent_{i}")
        dot.edge(f"sub_agent_{i}", NODE_IDS["collect_results"])
    dot.edge(NODE_IDS["collect_results"], NODE_IDS["combine_results"])
    dot.edge(NODE_IDS["combine_results"], NODE_IDS["final_output"])
    dot.edge(NODE_IDS["simple_responder"], NODE_IDS["final_output"])

    output_path = os.path.join(temp_dir, 'graph')
    dot.render(output_path, format='png', cleanup=True)
    return f"{output_path}.png"



def run_subtask(subtask: str, worker_agent: autogen.AssistantAgent) -> str:
    """
    Helper to run a single sub-task with all tools registered.
    This version is robust against agents ending on a tool call.
    """
    print(f"  🧵 [Thread] Starting task for {worker_agent.name}: '{subtask}'")
    thread_proxy = autogen.UserProxyAgent(name=f"Proxy_{worker_agent.name}", human_input_mode="NEVER", code_execution_config=False)

    # Register all available tools for the sub-agent
    autogen.register_function(
        duckduckgo_search,
        caller=worker_agent, executor=thread_proxy, name="duckduckgo_search",
        description="Performs a web search using DuckDuckGo. Use for general-purpose searches, current events, or when other tools fail."
    )
    autogen.register_function(
        wikipedia_search,
        caller=worker_agent, executor=thread_proxy, name="wikipedia_search",
        description="Fetches a concise summary from Wikipedia. Best for well-defined entities like people, places, organizations, or historical events."
    )
    autogen.register_function(
        langsearch_web_search,
        caller=worker_agent, executor=thread_proxy, name="langsearch_web_search",
        description="Performs a web search using the LangSearch API to get a summarized list of results. Good for broad research queries."
    )

    thread_proxy.initiate_chat(worker_agent, message=subtask, max_turns=5, silent=False)

    # --- ROBUSTNESS FIX STARTS HERE ---
    final_message = thread_proxy.last_message(worker_agent)

    # Check if a final message exists and has content.
    if final_message and isinstance(final_message.get("content"), str) and final_message["content"].strip():
        result = final_message["content"]
        print(f"  ✅ [Thread] Finished task for {worker_agent.name}.")
        return result
    else:
        # If the agent ends on a tool call or produces no content, provide a default response.
        print(f"  ⚠️ [Thread] Sub-agent {worker_agent.name} did not produce a final text response. Returning a default message.")
        return f"Sub-agent {worker_agent.name} completed its search for '{subtask}' but did not provide a summary."
    # --- ROBUSTNESS FIX ENDS HERE ---

def run_full_process(user_query: str):
    """The main generator function that runs the whole process and yields UI updates."""
    log = ""
    statuses = {node_id: STATUS_COLORS['pending'] for node_id in NODE_IDS.values()}
    statuses[NODE_IDS["human"]] = STATUS_COLORS['human']

    # --- Initial State ---
    log += "Process started. Waiting for user input...\n"
    yield create_graph_image(statuses), log

    # --- Phase 1: Classification ---
    statuses[NODE_IDS["classifier"]] = STATUS_COLORS['running']
    log += f"\n--- [Phase 1/2] Classifying Query ---\nQuery: '{user_query}'\n"
    yield create_graph_image(statuses), log
    time.sleep(1)

    labels = ["trivial", "non-trivial", "greetings", "alphabets","letters","symbols","numbers","greetings","research","technology"]
    best_label_info = classify_query(CLASSIFIER_PIPELINE, user_query, labels)
    best_label = best_label_info['label']
    statuses[NODE_IDS["classifier"]] = STATUS_COLORS['completed']
    log += f"📊 Query classified as: '{best_label}' with score {best_label_info['score']:.2f}.\n"
    yield create_graph_image(statuses), log
    time.sleep(1)

    # --- Phase 2: Execution (Two Paths) ---
    if best_label in ["trivial", "greetings","alphabets","letters","symbols","numbers"]:
        # --- SIMPLE WORKFLOW ---
        log += "\n--- [Phase 2/2] Executing Simple Q&A Workflow ---\n"
        statuses[NODE_IDS["simple_responder"]] = STATUS_COLORS['running']
        yield create_graph_image(statuses), log

        simple_responder = autogen.AssistantAgent(name="SimpleResponder", llm_config=llm_config, system_message="You are a helpful AI assistant. Answer the user's query directly and concisely. If the query is a greeting, respond politely.")
        proxy = autogen.UserProxyAgent(name="UserProxy", human_input_mode="NEVER")
        proxy.initiate_chat(simple_responder, message=user_query, max_turns=1, silent=True)
        final_response = proxy.last_message(simple_responder)["content"]

        statuses[NODE_IDS["simple_responder"]] = STATUS_COLORS['completed']
        statuses[NODE_IDS["final_output"]] = STATUS_COLORS['running']
        yield create_graph_image(statuses), log
        time.sleep(1)
        statuses[NODE_IDS["final_output"]] = STATUS_COLORS['completed']
        log += f"\n✅ FINAL RESPONSE:\n\n{final_response}\n"
        yield create_graph_image(statuses), log

    else:
        # --- COMPLEX WORKFLOW ---
        num_subtasks = 0

        # Coordinator
        statuses[NODE_IDS["coordinator_receives"]] = STATUS_COLORS['running']
        log += "\n--- [Phase 2/6] Coordinator: Breaking down task... ---\n"
        yield create_graph_image(statuses, num_subtasks), log
        coordinator = autogen.AssistantAgent(name="Coordinator", llm_config=llm_config, system_message=f"You are a master coordinator. Break down the user's query into 4 smaller, independent sub-tasks for web research. Respond with ONLY a Python-parseable list of strings.\nUser Query: \"{user_query}\"")
        proxy = autogen.UserProxyAgent(name="MainProxy", human_input_mode="NEVER")
        proxy.initiate_chat(coordinator, message=f"Break down this query: {user_query}", max_turns=1, silent=True)
        subtasks_response = proxy.last_message(coordinator)["content"]
        statuses[NODE_IDS["coordinator_receives"]] = STATUS_COLORS['completed']
        yield create_graph_image(statuses, num_subtasks), log

        # Subtask List Creation
        statuses[NODE_IDS["subtasks_list"]] = STATUS_COLORS['running']
        yield create_graph_image(statuses, num_subtasks), log
        subtasks = []
        try:
            list_string = extract_python_list_from_string(subtasks_response)
            subtasks = ast.literal_eval(list_string) if list_string else [user_query]
        except:
            subtasks = [user_query]
        num_subtasks = len(subtasks)
        log += f"✅ Coordinator generated {num_subtasks} subtasks: {subtasks}\n"
        statuses[NODE_IDS["subtasks_list"]] = STATUS_COLORS['completed']
        yield create_graph_image(statuses, num_subtasks), log

        # Create & Execute Sub-agents
        statuses[NODE_IDS["create_subagents"]] = STATUS_COLORS['running']
        log += "\n--- [Phase 3/6] Creating & Executing Sub-agents... ---\n"
        yield create_graph_image(statuses, num_subtasks), log
        time.sleep(1)
        statuses[NODE_IDS["create_subagents"]] = STATUS_COLORS['completed']
        statuses[NODE_IDS["execute_parallel"]] = STATUS_COLORS['running']
        for i in range(1, num_subtasks + 1): statuses[f"sub_agent_{i}"] = STATUS_COLORS['running']
        yield create_graph_image(statuses, num_subtasks), log

        # UPDATED System message for worker agents
        worker_system_message = """You are a research agent. You have access to three tools:
1. `wikipedia_search`: Best for well-defined topics (e.g., "Eiffel Tower", "Albert Einstein").
2. `duckduckgo_search`: Best for general web searches, opinions, or very current events.
3. `langsearch_web_search`: Best for broad research questions that benefit from summarized results from multiple sources.

Based on your given sub-task, choose the single most appropriate tool, execute it, and then formulate a comprehensive answer based ONLY on the tool's output. Respond with the final answer directly."""

        worker_agents = [autogen.AssistantAgent(name=f"Sub_Agent_{i+1}", llm_config=llm_config, system_message=worker_system_message) for i in range(num_subtasks)]
        results = []
        with concurrent.futures.ThreadPoolExecutor(max_workers=num_subtasks) as executor:
            future_to_task = {executor.submit(run_subtask, task, agent): task for agent, task in zip(worker_agents, subtasks)}
            for future in concurrent.futures.as_completed(future_to_task):
                try: results.append(future.result())
                except Exception as e: results.append(f"Error processing subtask: {e}")

        log += "✅ All sub-agent threads have completed.\n"
        statuses[NODE_IDS["execute_parallel"]] = STATUS_COLORS['completed']
        for i in range(1, num_subtasks + 1): statuses[f"sub_agent_{i}"] = STATUS_COLORS['completed']
        yield create_graph_image(statuses, num_subtasks), log

        # Collect & Combine Results
        for node_id, phase_name in [(NODE_IDS["collect_results"], "Collecting"), (NODE_IDS["combine_results"], "Synthesizing")]:
            statuses[node_id] = STATUS_COLORS['running']
            log += f"\n--- [Phase {4 if node_id == NODE_IDS['collect_results'] else 5}/6] {phase_name} Results... ---\n"
            yield create_graph_image(statuses, num_subtasks), log
            time.sleep(1)
            statuses[node_id] = STATUS_COLORS['completed']
            yield create_graph_image(statuses, num_subtasks), log

        # Final Output
        statuses[NODE_IDS["final_output"]] = STATUS_COLORS['running']
        log += "\n--- [Phase 6/6] Generating Final Output... ---\n"
        yield create_graph_image(statuses, num_subtasks), log
        synthesizer = autogen.AssistantAgent(name="Synthesizer", llm_config=llm_config, system_message="You are a master report writer. Synthesize results from sub-agents into a single, cohesive, final answer.")
        combined_results = "\n\n".join(results)
        synthesis_message = f"Original query: \"{user_query}\"\n\nResults from sub-tasks:\n{combined_results}\n\nSynthesize these into a final, well-structured answer."
        proxy.initiate_chat(synthesizer, message=synthesis_message, max_turns=1, silent=True)
        final_response = proxy.last_message(synthesizer)["content"]

        statuses[NODE_IDS["final_output"]] = STATUS_COLORS['completed']
        log += f"\n✅ FINAL RESPONSE:\n\n{final_response}\n"
        yield create_graph_image(statuses, num_subtasks), log

In [ ]:
# Cell 5: Build and Launch the Gradio UI

with gr.Blocks(theme=gr.themes.Soft(), title="Live Agentic Workflow") as demo:
    gr.Markdown("# Live Visualization of an AutoGen Agentic Workflow")
    gr.Markdown("Enter a query below. The system will classify it, choose a workflow (Simple Q&A or Complex Research), and visualize the process live.")

    with gr.Row():
        query_input = gr.Textbox(
            label="Your Query",
            placeholder="e.g., What were the key findings of the Llama 3 paper?",
            scale=3
        )
        start_button = gr.Button("▶️ Start Process", variant="primary", scale=1)

    with gr.Row():
        graph_output = gr.Image(label="Process Flowchart", type="filepath", height=700, interactive=False)
        log_output = gr.Textbox(label="Process Log & Final Answer", lines=25, interactive=False)

    # Connect the button to the main workflow function
    start_button.click(
        fn=run_full_process,
        inputs=[query_input],
        outputs=[graph_output, log_output]
    )

    # Add examples for users to try
    gr.Examples(
        examples=[
            "Hello, how are you?",
            "What is the capital of France?",
            "Compare the pros and cons of using React vs. Vue for a new web project.",
            "What are the latest advancements in treating Alzheimer's disease?",
        ],
        inputs=query_input
    )

# Launch the app - share=True is essential for Google Colab
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://82d40337530c776972.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


100%|██████████| 1/1 [00:01<00:00,  1.62s/it]



>>>>>>>> TERMINATING RUN (67bd9882-b780-461e-897b-b15b870b46e8): Maximum turns (1) reached


/usr/local/lib/python3.11/dist-packages/autogen/oai/groq.py:303: UserWarning: Cost calculation not available for model meta-llama/llama-4-maverick-17b-128e-instruct
  warnings.warn(f"Cost calculation not available for model {model}", UserWarning)


  🧵 [Thread] Starting task for Sub_Agent_1: 'Identify pros of using React for web development'
Proxy_Sub_Agent_1 (to Sub_Agent_1):

Identify pros of using React for web development
  🧵 [Thread] Starting task for Sub_Agent_2: 'Identify cons of using React for web development'

--------------------------------------------------------------------------------
  🧵 [Thread] Starting task for Sub_Agent_3: 'Identify pros of using Vue for web development'
Proxy_Sub_Agent_2 (to Sub_Agent_2):

  🧵 [Thread] Starting task for Sub_Agent_4: 'Identify cons of using Vue for web development'
Identify cons of using React for web development

--------------------------------------------------------------------------------
Proxy_Sub_Agent_3 (to Sub_Agent_3):

Identify pros of using Vue for web development

--------------------------------------------------------------------------------
Proxy_Sub_Agent_4 (to Sub_Agent_4):

Identify cons of using Vue for web development

-------------------------------------

/usr/local/lib/python3.11/dist-packages/autogen/oai/groq.py:303: UserWarning: Cost calculation not available for model meta-llama/llama-4-maverick-17b-128e-instruct
  warnings.warn(f"Cost calculation not available for model {model}", UserWarning)



🌐 [Sub-agent Tool] Searching LangSearch for: 'cons of using Vue for web development'
Proxy_Sub_Agent_3 (to Sub_Agent_3):

***** Response from calling tool (d1bvay0fd) *****
Error during LangSearch for 'pros of using Vue for web development': 429 Client Error:  for url: https://api.langsearch.com/v1/web-search
**************************************************

--------------------------------------------------------------------------------
Proxy_Sub_Agent_2 (to Sub_Agent_2):

***** Response from calling tool (9sxqxvq2v) *****
Proxy_Sub_Agent_1 (to Sub_Agent_1):

***** Response from calling tool (82yzpewm7) *****
Error during LangSearch for 'cons of using React for web development': 429 Client Error:  for url: https://api.langsearch.com/v1/web-search
Error during LangSearch for 'pros of using React for web development': 429 Client Error:  for url: https://api.langsearch.com/v1/web-search
**************************************************
************************************************

  ✅ [Thread] Finished task for Sub_Agent_3.



>>>>>>>> TERMINATING RUN (55fb8755-bb0b-4aff-b2e0-77bda7fea8f5): Maximum turns (1) reached


/usr/local/lib/python3.11/dist-packages/autogen/oai/groq.py:303: UserWarning: Cost calculation not available for model meta-llama/llama-4-maverick-17b-128e-instruct
  warnings.warn(f"Cost calculation not available for model {model}", UserWarning)
